In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load and preprocess the data

In [17]:
data_dir = "../data/images_train/"

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40, # these parameters can be tuned
    width_shift_range=0.2, # these parameters can be tuned
    height_shift_range=0.2, # these parameters can be tuned
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16, # batch size can be tuned
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation' 
)

Found 493 images belonging to 2 classes.
Found 123 images belonging to 2 classes.


# Load a pre-trained ResNet50 model

In [18]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [19]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x) # dropout can be tuned
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model

In [21]:
model.compile(optimizer=Adam(learning_rate=0.001), # learning rate can be tuned
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Train the model

In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10, # epochs can be tuned
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[early_stopping]
)

Epoch 1/10
30/30 [==============================] - 76s 2s/step - loss: 1.0090 - accuracy: 0.5472 - val_loss: 0.6111 - val_accuracy: 0.6875
Epoch 2/10
30/30 [==============================] - 69s 2s/step - loss: 0.6391 - accuracy: 0.6583 - val_loss: 0.5920 - val_accuracy: 0.6875
Epoch 3/10
30/30 [==============================] - 71s 2s/step - loss: 0.7058 - accuracy: 0.6247 - val_loss: 0.6620 - val_accuracy: 0.6250
Epoch 4/10
30/30 [==============================] - 66s 2s/step - loss: 0.6135 - accuracy: 0.6792 - val_loss: 0.5858 - val_accuracy: 0.7500
Epoch 5/10
30/30 [==============================] - 67s 2s/step - loss: 0.6103 - accuracy: 0.6855 - val_loss: 0.6573 - val_accuracy: 0.6161
Epoch 6/10
30/30 [==============================] - 69s 2s/step - loss: 0.5688 - accuracy: 0.7317 - val_loss: 0.5559 - val_accuracy: 0.7321
Epoch 7/10
30/30 [==============================] - 67s 2s/step - loss: 0.5783 - accuracy: 0.7170 - val_loss: 0.5947 - val_accuracy: 0.7143
Epoch 8/10
30/30 [==

Adam, learning rate 0.0001 val_accuracy: 0.6042

batch = 16, Adam, learning rate 0.001 accuracy 0.7321

In [23]:
test_dir = '../data/images_test'

predictions = []

for image_name in os.listdir(test_dir):
    image_id = image_name[:3]
    img_path = os.path.join(test_dir, image_name)
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array /= 255.

    # predict
    prediction = model.predict(img_array)
    predicted_class = int(prediction[0][0] <= 0.5) # 0 for kitchen, 1 for bathroom
    predictions.append([image_id,predicted_class])

predictions_df = pd.DataFrame(predictions, columns=['id', 'class'])
predictions_df.to_csv('image_prediction.csv', index=False)

1/1 [==============================] - 0s 92ms/step


In [24]:
predictions_df

,id,class
0,100,0
1,101,0
2,102,1
3,103,1
4,104,1
...,...,...
144,269,1
145,270,0
146,271,0
147,272,1


In [25]:
# train_generator.class_indices